In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
from datasets import load_dataset, load_metric, Dataset

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import pickle
import pandas as pd
from datasets import Value
from torch.utils.data import DataLoader
import argparse
import os
from pymongo import MongoClient
import IPython
from tqdm import tqdm

In [2]:
client = MongoClient()
db_metoo_tweets = client["jason_twitter"]
metoo_tweets = db_metoo_tweets.tweets

In [3]:
tweet_cursor = metoo_tweets.find({"condemnation_prediction":1})

In [4]:
checkpoint = "bert-base-uncased"
model_path = "./models/hf"

# model = torch.load(model_path)
# model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [5]:
def split_list(cursor, n):
        result = []
        for tweet in tqdm(cursor):
            result.append(tweet)
            if len(result)==n:
                result_to_return = result
                result = []
                yield result_to_return
        yield result
        
def update_tweet_in_db( document):
    try:
        metoo_tweets.update_one(
            {'_id': document['_id']},
            {'$set': document}
        )
    except Exception:
        print("couldn't update ", document)

In [6]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="exp/bart/results",
    do_train=False,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=1000,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=128,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=1,
)
# training_args = TrainingArguments("test-trainer")
# training_args.eval_accumulation_steps = 1  # pushes predictions out of GPU to mitigate GPU out of memory

trainer = Trainer(
    model = AutoModelForSequenceClassification.from_pretrained("models/hf/hf_fold_1_model"),
    args=training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [7]:
n=100000
target_col = "clean_tweet"
for idx, chunk in tqdm(enumerate(split_list(tweet_cursor, n))):
        df = pd.DataFrame(chunk)
        if not len(df):
            break
        pred_df = df[[target_col]].dropna()
        pred_data = df.dropna(subset=[target_col])
        pred_dataset = Dataset.from_pandas(pred_df)
        pred_dataset = pred_dataset.rename_column(target_col, "text")
        tokenized_datasets = pred_dataset.map(tokenize_function, batched=True)
        predictions_logits = trainer.predict(tokenized_datasets)
        preds = np.argmax(predictions_logits.predictions, axis = 1)
        save_df = pred_data[["_id", "clean_tweet"]]
        save_df["severity_prediction"] = preds
        save_df["severity_logit_0"] = predictions_logits.predictions[:, 0]
        save_df["severity_logit_1"] = predictions_logits.predictions[:, 1]
        save_df["severity_logit_2"] = predictions_logits.predictions[:, 1]
    
        del tokenized_datasets
        del pred_dataset
        for idx, row in tqdm(save_df.iterrows()):
            update_tweet_in_db(row.to_dict())
        

0it [00:00, ?it/s]
0it [00:00, ?it/s]
102it [00:00, 202.78it/s]
3547it [00:00, 4432.55it/s]
6638it [00:01, 5347.33it/s]
9732it [00:01, 5424.25it/s]
12829it [00:02, 5137.05it/s]
15939it [00:02, 6529.95it/s]
19167it [00:03, 5614.78it/s]
22427it [00:03, 6983.99it/s]
25701it [00:04, 5510.60it/s]
28961it [00:04, 6563.17it/s]
32151it [00:06, 4877.17it/s]
35338it [00:06, 5876.55it/s]
38617it [00:06, 6991.97it/s]
41889it [00:07, 4592.49it/s]
45149it [00:08, 5653.51it/s]
48319it [00:08, 6901.47it/s]
51401it [00:08, 8135.21it/s]
54609it [00:10, 4411.87it/s]
57887it [00:10, 5607.77it/s]
61166it [00:10, 6955.23it/s]
64517it [00:10, 8408.13it/s]
67957it [00:10, 9890.17it/s]
71353it [00:12, 4396.49it/s]
74757it [00:12, 5657.45it/s]
78113it [00:13, 7045.77it/s]
81474it [00:13, 8509.05it/s]
84890it [00:13, 10040.10it/s]
88266it [00:13, 11390.57it/s]
91656it [00:15, 3943.80it/s] 
95027it [00:16, 5103.85it/s]
98387it [00:16, 6406.42it/s]

  0%|          | 0/100 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100000
  Batch size = 128



98387it [00:30, 6406.42it/s]/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to b

73079it [00:42, 1829.45it/s]

73269it [00:42, 1849.85it/s]

73455it [00:42, 1850.07it/s]

73641it [00:42, 1789.57it/s]

73824it [00:42, 1798.86it/s]

74011it [00:42, 1817.88it/s]

74194it [00:42, 1807.24it/s]

74379it [00:42, 1818.89it/s]

74566it [00:42, 1833.56it/s]

74753it [00:42, 1843.48it/s]

74942it [00:43, 1855.81it/s]

75128it [00:43, 1849.45it/s]

75313it [00:43, 1847.46it/s]

75499it [00:43, 1848.89it/s]

75684it [00:43, 1843.52it/s]

75872it [00:43, 1853.89it/s]

76060it [00:43, 1859.32it/s]

76247it [00:43, 1862.47it/s]

76437it [00:43, 1871.39it/s]

76627it [00:43, 1877.24it/s]

76815it [00:44, 1868.54it/s]

77002it [00:44, 1864.07it/s]

77189it [00:44, 1826.41it/s]

77375it [00:44, 1834.04it/s]

77561it [00:44, 1841.05it/s]

77752it [00:44, 1861.03it/s]

77940it [00:44, 1865.58it/s]

78127it [00:44, 1799.80it/s]

78308it [00:44, 1774.64it/s]

78487it [00:44, 1777.14it/s]

78665it [00:45, 1773.25it/s]

78843it [00:45, 1737.64it/s]

79018it [00:45, 1650.03it/s]

79184it [0

...

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100000
  Batch size = 128

8198078it [5:41:41, 6394.83it/s]/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

74939it [00:40, 1834.73it/s]

75123it [00:40, 1825.48it/s]

75306it [00:40, 1820.04it/s]

75489it [00:40, 1790.56it/s]

75673it [00:41, 1803.75it/s]

75854it [00:41, 1802.78it/s]

76039it [00:41, 1816.56it/s]

76228it [00:41, 1837.52it/s]

76413it [00:41, 1839.16it/s]

76599it [00:41, 1842.14it/s]

76787it [00:41, 1852.45it/s]

76976it [00:41, 1863.34it/s]

77166it [00:41, 1873.77it/s]

77354it [00:41, 1873.90it/s]

77542it [00:42, 1797.39it/s]

77723it [00:42, 1655.10it/s]

77915it [00:42, 1726.35it/s]

78102it [00:42, 1764.95it/s]

78287it [00:42, 1788.74it/s]

78477it [00:42, 1820.86it/s]

78668it [00:42, 1845.61it/s]

78859it [00:42, 1863.09it/s]

79046it [00:42, 1856.29it/s]

79238it [00:43, 1873.47it/s]

79426it [00:43, 1867.72it/s]

79616it [00:43, 1875.56it/s]

79806it [00:43, 1882.56it/s]

79995it [00:43, 1883.08it/s]

80184it [00:43, 1880.76it/s]

80373it [00:43, 1882.07it/s]

80563it [00:43, 1885.19it/s]

80752it [00:43, 1876.35it/s]

80943it [00:43, 1885.60it/s]

81132it [0

  0%|          | 0/58 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 57732
  Batch size = 128
/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

36620it [00:20, 1822.05it/s]
36803it [00:20, 1801.68it/s]
36990it [00:20, 1820.73it/s]
37173it [00:20, 1817.77it/s]
37359it [00:20, 1827.91it/s]
37544it [00:20, 1833.81it/s]
37728it [00:20, 1800.20it/s]
37909it [00:20, 1777.89it/s]
38087it [00:20, 1759.60it/s]
38264it [00:20, 1760.09it/s]
38449it [00:21, 1783.92it/s]
38628it [00:21, 1721.65it/s]
38801it [00:21, 1722.16it/s]
38974it [00:21, 1706.76it/s]
39145it [00:21, 1607.58it/s]
39307it [00:21, 1419.09it/s]
39471it [00:21, 1476.49it/s]
39635it [00:21, 1519.29it/s]
39809it [00:21, 1578.36it/s]
39989it [00:22, 1640.69it/s]
40170it [00:22, 1687.76it/s]
40352it [00:22, 1722.92it/s]
40530it [00:22, 1737.35it/s]
40705it [00:22, 1735.71it/s]
40880it [00:22, 1732.48it/s]
41054it [00:22, 1688.52it/s]
41235it [00:22, 1721.64it/s]
41417it [00:22, 1750.04it/s]
41602it [00:22, 1776.71it/s]
41787it [00:23, 1796.49it/s]
41976it [00:23, 1821.61it/s]
42166it [00:23, 1844.35it/s]
42351it [00:23, 1838.01it/s]
42535it [00:23, 1836.66it/s]
42719it [00:23

In [25]:
save_df = pred_data[["_id", "clean_tweet"]]
save_df["severity_prediction"] = preds
save_df["severity_logit_0"] = predictions_logits.predictions[:, 0]
save_df["severity_logit_1"] = predictions_logits.predictions[:, 1]
save_df["severity_logit_2"] = predictions_logits.predictions[:, 1]

/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/geev/Research/Transgression/tenv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [26]:
save_df

,_id,clean_tweet,severity_prediction,severity_logit_0,severity_logit_1,severity_logit_2
0,6383e2adfa2b796ff3841c3f,<TWEET>: The latest #Predator movie raised a l...,1,-1.418743,0.922649,0.922649
1,6383e2adfa2b796ff3841c46,<TWEET>: Don't cast your child molesting pedop...,2,-1.805397,0.236971,0.236971
2,6383e2adfa2b796ff3841c4b,<TWEET>: I just saw #ThePredator’s promoted tw...,1,-1.350736,0.753730,0.753730
3,6383e2adfa2b796ff3841c4c,<TWEET>: Perhaps if Steven Wilder Striegel had...,1,-0.378258,0.814329,0.814329
4,6383e2adfa2b796ff3841c4e,"<TWEET>: The former teen known as Jane Doe, no...",1,-0.725376,1.061773,1.061773
...,...,...,...,...,...,...
99995,6383e319fa2b796ff38a1f27,"<TWEET>: Wait, so Kevin Spacey's response to b...",1,-0.849389,0.973997,0.973997
99996,6383e319fa2b796ff38a1f29,<TWEET>: I see Kevin Spacey has come out as a ...,1,-1.355826,0.965941,0.965941
99997,6383e319fa2b796ff38a1f2e,<TWEET>: Why does Kevin Spacey have to be ruin...,1,-0.044263,0.708245,0.708245
99998,6383e319fa2b796ff38a1f31,<TWEET>: Bingo!(this is the 1 that came forwar...,1,-1.326478,1.006863,1.006863


In [24]:
preds[:400]

array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1,
       1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2,
       2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2,
       1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 2,
       2, 2, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2,
       1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 0, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2,
       2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1,
       2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 1,

In [14]:
predictions_logits

PredictionOutput(predictions=array([[-1.4187431 ,  0.9226494 , -0.09052195],
       [-1.8053972 ,  0.23697093,  1.5364958 ],
       [-1.3507361 ,  0.7537301 ,  0.0857865 ],
       ...,
       [-0.04426257,  0.7082449 , -0.8602782 ],
       [-1.3264781 ,  1.0068629 ,  0.06774779],
       [-0.56836873,  0.97940767, -0.70287985]], dtype=float32), label_ids=None, metrics={'test_runtime': 169.983, 'test_samples_per_second': 588.294, 'test_steps_per_second': 2.3})

In [4]:
file = open('../../data/5_mil_7days_metoo.p', 'rb')
pred_data = pickle.load(file)
file.close()
pred_df = pred_data[["clean_tweet_masked"]].dropna()
pred_df

,clean_tweet_masked
0,"<TWEET>: » “I do love you. . I love you, <TARG..."
4,<TWEET>: <TARGET 1> apologizes for 'aggressive...
5,<TWEET>: No one should have to endure this kin...
6,"<TWEET>: ""New <TARGET 1> sexual assault accusa..."
8,"<TWEET>: Yes this. <TARGET 1>, this clown, the..."
...,...
4683897,<TWEET>: On the one year anniversary of the Ac...
4683898,<TWEET>: <TARGET 1>'s photobombing. The ladies...
4683900,<TWEET>: But but but dude you're a rapist HOW ...
4683901,"<TWEET>: Lisa Bloom, Lawyer Advising <TARGET 1..."


In [6]:
pred_df = pred_df.head(1000)

In [7]:
from datasets import Value
pred_dataset = Dataset.from_pandas(pred_df)
pred_dataset = pred_dataset.rename_column("clean_tweet_masked", "text")

In [8]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)
tokenized_datasets = pred_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [9]:
model_path = "./sample_model.p"
model = torch.load(model_path)
model.eval()
# model.to("cpu")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [13]:
from transformers import Trainer
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="exp/bart/results",
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=100,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=1,
)
training_args = TrainingArguments("test-trainer")
training_args.eval_accumulation_steps=1 #pushes predictions out of GPU to mitigate GPU out of memory

trainer = Trainer(
    model,
    args=args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [15]:
predictions = trainer.predict(tokenized_datasets)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, __index_level_0__.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


In [ ]:
with open('./severity_predictions', 'wb') as f:
    pickle.dump(predictions, f)